In [25]:
%pip install pandas # python-dotenv transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [29]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from huggingface_hub import login
from dotenv import load_dotenv

import pandas as pd

import torch
import os

In [13]:
# Load environment variables from .env file
load_dotenv()

token = os.getenv("HUGGINGFACE_API_TOKEN")

login(token)

In [ ]:
model_id = "meta-llama/Llama-3.2-1B"

pipe = pipeline(
    "text-generation", 
    model=model_id, 
    torch_dtype=torch.bfloat16,
    max_length=50
)

response = pipe("What is the most common eaten foodstuff in USA, Los Angeles?")

response

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'What is the most common eaten foodstuff in USA, Los Angeles? The most common food eaten in USA is pizza, followed by hamburger, and then french fries. What is the most common food eaten in Los Angeles? The most common food eaten in Los Angeles is pizza, followed by hamburger, and then french fries. What is the most common food eaten in Los Angeles? The most common food eaten in Los Angeles is pizza, followed by hamburger, and then french fries. What is the most'}]

In [ ]:
# Finetuning with our data

# 1. load pretrained Tokenizer, call it with dataset (encode the dataset)
model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

sequence = "This is temporary tokenized dataset"
tokenized_dataset = tokenizer(sequence, return_tensors="pt")
print(tokenized_dataset)

# training_args = TrainingArguments("trainer")

# trainer = Trainer(
#     model,
#     training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["validation"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
# )

# trainer.train()

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[128000,   2028,    374,  13643,   4037,   1534,  10550]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# Fine-tuned model

classifier = pipeline(
    "text-generation", 
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    max_length=50
)

In [ ]:
# Save model
model.save_pretrained("meta-llama/Llama-3.2-1B")

# Load model
model = AutoModelForSequenceClassification.from_pretrained("meta-llama/Llama-3.2-1B")